# Стохастические методы оптимизации

По большому счету, все методы оптимизации можно разделить на два больших класса: детерминированные и стохастические.

Разница между ними заключается в том, что в детерминированных методах на каждом шаге точка, в которую мы перейдем на следующем шаге, вычисляется однозначно, по формуле или алгоритму, а в стохастических методах выбирается случайно каким-либо образом.

В первой работе мы с вами познакомились с обоими подходами: метод покоординатного спуска - классический представитель детерминированных методов, а метод случайного поиска, как следует из его названия - представитель класса стохастических методов.

Казалось бы, детерминированные методы предпочтительнее, ведь они всегда выбирают "наилучшее" решение на каждом шаге?

К сожалению, они идеально подходят только для задач с гладкими монотонно возрастающими или убывающими целевыми функциями. Если целевая функция имеет локальные минимумы или максимумы, есть большая вероятность, что алгоритм оптимизации остановится, достигнув именно локального максимума или минимума, но не найдет глобальный.

Для задач, где целевая функция имеет сложную форму, стохастические методы зачастую оказываются более подходящими.

Одним из типичных представителей стохастических методов является *метод отжига*.

## Метод отжига

Этот метод был разработан в 1953, причем основа метода пришла из обработки металлов.

В то время перед металлургами стояла задача получения качественных, прочных металлических изделий. На тот момент было известно, что прочность, среди прочих факторов, определяется также структурой металла, а именно, образовали ли молекулы металла кристаллическую решетку в процессе остывания из расплава.

Понятно, что в жидком металле кристаллическая решетка отсутствует, и молекулы могу свободно перемещаться, а в застывшем металле молекулы перемещаться не могут, но от того, встали ли они на "правильное" место, на свое место в кристаллической решетке, зависит прочность готового изделия.

В общем-то, молекулы металла стремятся занять свои места в решетке, поскольку именно в этом случае энергия системы будет наименьшей (собственно, именнно поэтому и прочность изделия в этом случае выше, так как в металле отстутствуют точки напряжения), но не всегда "успевают" это сделать в процессе остывания металла.

Занять свои места они могут, свободно двигаясь в расплаве (при этом с большей вероятностью молекула перейдет в  точку, соответствующую месту в кристаллической решетке), но по мере того, как температура расплава уменьшается, подвижность молекул металла падает. Поэтому металлурги повышали качество металлических отливок, добиваясь *постепенного* и *плавного* уменьшения температуры, чтобы все (или хотя бы большинство) молекул металла успевали занять свои места в кристаллической структуре.

Собственно, метод отжига в оптимизации заимствует идеи из металлургии.

В нашем случае энергии системы соответствует значение целевой функции, молекулам металла соответствует текущая точка в пространстве состояний системы (текущие значения переменных $x$), температура соответствует некоторой величине, которая определяет, как далеко от текущей точки может находиться следующая, в которую мы перейдем, причем температура падает с каждым шагом. Процесс останавливается, когда температура достигнет нуля.

Таким образом, алгоритм включает в себя следующие элементы:

* текущая точка $x$
* значение целевой функции в этой точке $f(x)$
* текущее значение "температуры" $T$
* функция уменьшения температуры на каждом шаге
* функция выбора следующей точки $x^n = G(x)$
* функция вычисления вероятности перехода в следующую точку в зависимости от разницы значений целевой функции в новой и старой точках и текущей температуры $h(f(x^n) - f(x), T)$

Алгоритм состоит из следующих шагов:

1. Выбрать начальную точку $x$, начальную температуру $T$, задать границу вероятности перехода в новую точку $P$
2. Пока температура $T$ больше нуля, повторять:
    1. Вычислить значение целевой функции $f(x)$ в текущей точке $x$
    2. Случайным образом выбрать смещение от текущей точки $x$, и получить новую точку $x^n$
    3. Вычислить значение целевой функции в новой точке $f(x^n)$
    4. Вычислить вероятность перехода в новую точку $h = h(f(x^n) - f(x), T)$
    5. Если вычисленная вероятность $h$ больше заданной $P$, перейти в новую точку, иначе повторить шаг 2.B.
    6. понизить темепературу

Алгоритм останавливается, когда значение темепературы достигнет нуля.

Использование алгоритма не гаратирует нахождения глобального оптимума, но сам по себе алгоритм являет довольно эффективным.

Рассмотрим метод отжига на примере:

In [6]:
import math
import random

# Это сам метод отжига
# в качестве параметров передаются начальное значение x, начальное значение температуры t,
# функция, уменьшающая значение температуры на каждом шаге decrease_temperature, граница вероятности p, минимизируемая функция f,
# функция генерации нового значения x generate_new_x, функция вычисления вероятности перехода в новую точку h
# в качестве результата функция возвращает найденное значение x, при котором функция f минимальна, и само значение этой функции f(x)
def annealing(x: float, temperature: float, decrease_temperature: callable, p: float, f: callable, generate_new_x: callable, h: callable) -> (float, float):
    while temperature > 0:
        target_function_value = f(x)
        print(f"x = {x}, f(x)={target_function_value}")
        while True:
            x_n = generate_new_x(x)
            new_target_function_value = f(x_n)

            new_probability = h(new_target_function_value - target_function_value, temperature)

            if new_probability > p:
                print(f"x = {x}, f(x) = {target_function_value}, x_new = {x_n}, f(x) = {new_target_function_value}, prob = {new_probability}")

                x = x_n
                target_function_value = new_target_function_value
                break
        temperature = decrease_temperature(temperature)

    return x, f(x)

# это функция вычисления веряотности перехода в новую точку
def h_probability_function(delta_e: float, temperature: float) -> float:
    return math.exp(- delta_e / temperature)

# эта функция уменьшает темепературу на каждом шаге
def linear_decrease_temperature(current_temperature: float) -> float:
    return current_temperature - 1

# эта функция вычисляет новую точку на основе текущей (в данном случае случайно)
def generate_new_x(x: float) -> float:
    return x + random.random() - 0.5

# это функция, которую мы минимизируем
def func_to_minimize(x):
    return (x - 2) * (x - 2) + 1

In [7]:
# найдем минимум, используя метод имитации отжига
# с начальной точкой x = 0, начальной температурой 100 (то есть всего будет 100 шагов), с вероятностью перехода
# в новую точку 0.15
annealing(0, 100, linear_decrease_temperature, (1 - 0.15), func_to_minimize, generate_new_x, h_probability_function)

x = 0, f(x)=5
x = 0, f(x) = 5, x_new = -0.03670710978543412, f(x) = 5.148175851050535, prob = 0.9985193387516103
x = -0.03670710978543412, f(x)=5.148175851050535
x = -0.03670710978543412, f(x) = 5.148175851050535, x_new = 0.06342673689668943, f(x) = 4.7503160033666045, prob = 1.0040268724906076
x = 0.06342673689668943, f(x)=4.7503160033666045
x = 0.06342673689668943, f(x) = 4.7503160033666045, x_new = -0.43403136481671456, f(x) = 6.924508684911519, prob = 0.9780586517728812
x = -0.43403136481671456, f(x)=6.924508684911519
x = -0.43403136481671456, f(x) = 6.924508684911519, x_new = -0.06308529169682175, f(x) = 5.25632092081576, prob = 1.0173465457782955
x = -0.06308529169682175, f(x)=5.25632092081576
x = -0.06308529169682175, f(x) = 5.25632092081576, x_new = -0.12643468257405832, f(x) = 5.521724459253837, prob = 0.9972391978510498
x = -0.12643468257405832, f(x)=5.521724459253837
x = -0.12643468257405832, f(x) = 5.521724459253837, x_new = 0.3205510005824569, f(x) = 3.8205489416445872, pr

(1.0872148042228706, 1.8331768136298923)

Выше рассмотрен пример для поиска минимума функции, которую мы уже рассматривали, $f(x) = (x - 2)^2 + 1$.

Попробуйте запустить предыдущую ячейку несколько раз и убедитесь, что результат каждый раз несколько различается.

Попробуйте несколько различных видов функции уменьшения темепературы (например, увеличьте или уменьшите скорость,  с которой меняется температура, попробуйте нелинейное уменьшение, когда вначале темепература уменьшается быстро, а затем все медленнее и медленнее, или наоборот), оцените эффективность такого подхода.

Список дополнительной литературы:

[Алгоритм имитации отжига](https://ru.wikipedia.org/wiki/Алгоритм_имитации_отжига)

[Введение в оптимизацию. Имитация отжига](https://habr.com/ru/post/209610/)

[Метод отжига](https://www.math.spbu.ru/user/gran/sb1/lopatin.pdf)

Задание для самостоятельной работы:

Используйте целевую функцию из работы №2 (минимизация времени пересечения поля).
Подберите начально значение темепературы таким образом, чтобы поставленная задача была решена (возможно, его придется менять несколько раз)

In [8]:
import math
import numpy as np
import random

# Задаем исходные данные
h1 = 0.10  # 100 метров
h2 = 0.10  # 100 метров
l = 1.0  # 1000 метров

v1 = 40 / 3600  # 40 км/ч в км/с
v2 = 30 / 3600  # 30 км/ч в км/с

c1 = 0.115  # л/км
c2 = 0.15  # л/км

a1 = 0.5  # вес первого критерия, т.е. времени движения
a2 = 0.5  # вес второго критерия, т.е. расхода топлива

# Описываем функцию, которую хотим минимизировать
def func_to_minimize(x):
    if x < 0 or x > l:
        return float('inf')  # возвращаем бесконечность, если x вне допустимых границ

    t1 = math.sqrt(h1 * h1 + x * x) / v1  # время на первом участке
    t2 = math.sqrt(h2 * h2 + (l - x) * (l - x)) / v2  # время на втором участке

    fuel1 = c1 * (math.sqrt(h1 * h1 + x * x))  # расход топлива на первом участке
    fuel2 = c2 * (math.sqrt(h2 * h2 + (l - x) * (l - x)))  # расход топлива на втором участке

    total_time = t1 + t2
    total_fuel = fuel1 + fuel2

    return a1 * total_time + a2 * total_fuel  # целевая функция

# Реализация метода отжига
def simulated_annealing(starting_point, initial_temp, cooling_rate, max_iterations):
    current_x = starting_point
    current_energy = func_to_minimize(current_x)

    best_x = current_x
    best_energy = current_energy

    temp = initial_temp

    for iteration in range(max_iterations):
        # Генерация нового решения
        new_x = current_x + random.uniform(-0.1, 0.1)  # небольшое случайное изменение

        # Ограничиваем новое значение x в границах [0, l]
        new_x = max(0, min(new_x, l))

        new_energy = func_to_minimize(new_x)

        # Если новое решение лучше, принимаем его
        if new_energy < current_energy:
            current_x = new_x
            current_energy = new_energy
        else:
            # Если новое решение хуже, принимаем его с некоторой вероятностью
            acceptance_probability = np.exp((current_energy - new_energy) / temp)
            if random.random() < acceptance_probability:
                current_x = new_x
                current_energy = new_energy

        # Обновление лучшего решения
        if current_energy < best_energy:
            best_x = current_x
            best_energy = current_energy

        # Понижаем температуру
        temp *= cooling_rate

    return best_x, best_energy

# Параметры метода отжига
starting_point = 0.0
initial_temp = 1.0
cooling_rate = 0.99
max_iterations = 1000

# Запуск метода отжига
optimal_x, optimal_value = simulated_annealing(starting_point, initial_temp, cooling_rate, max_iterations)

# Вывод результата
print("Оптимальное значение x:", optimal_x)
print("Минимальное значение функции:", optimal_value)


Оптимальное значение x: 0.8880494950165201
Минимальное значение функции: 49.28401791045348
